<a href="https://colab.research.google.com/github/mao-code/SLFN_Regularization/blob/main/SLFN_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is based on the HW4 of New Learning Algorithm class (To handle overfitting)
Using regularizing module. Not just using weight_tuning_r

<img
width="500px"
src="https://lh3.googleusercontent.com/fife/APg5EOZ3hBraPY_Aa_TvGC-NZDWh5IpjnJf6kCwI3NKKrgO3kswHjrAtc32_iiwHx_OiEylmWEvnKGSpU_XK-kLtIHu2u0JBpLiBIlEe0wueEsaqs2_Xtxsl_udgl6jW61Z2965WnmZZYFla7zyEgu6JSmd9P9H62NhRHpZgdpX95J7kmPmPcTyUxJa_Ua6FXJ9J2b_yrgpPN3x_6eA8wm8x2S8xS0A_dmbZbkbC0j_X1VKhvirgLfQygawq0uaaIf5FRE16xrBuSwQ7pZoj2Eyk_GO1YKbFaXjb3lsj8j05TjPHdTYC0S-xflbgKANERDNe6tG3MttAnhgPi32wAqgdbLYNqea3cV6vLgEyK9feGTQpioYgPTREj_idBef-Tz9ztFPUsHInV4yk1A92wgfFI73_3dCEwkifWp5Amdx2zyN0kNkmrdntOy6cMt7-hcf0UydEJ8rNSL5Vd8kzNIicCg-UjXYvplti6VNMk1n5HY7SVupD2FL_c8UUKZVApkDzTeh71lDGL8i5A1H-QMQGcXfrZjzpCamrHiMDJRFZi5NIvJ3A4fx25PjbMVhePhgzrazM62vlXRWcHww-U4wCMBn1wh2WTQ6Aa3mN8LOAdiw8j7abWfTpEoABI18KoanH7xs2AWPTFdxPCXHmwg3XieaTlBPU943WiRmMRblI2cMDxVsiBojUQ-jwXAlsEQDfYt_bHvFlHQfu5j_byjHXjhEZjPcAu0pAdDIMrYDogp9o2toDjJHy2f9ziuMOYayncA_DP5EgTJLjgG5AVaOABRDGmV07ar5FxEEhXpk9E07KNSTiERq2g3g2SIwnalP8wP9d7yRfzNNRwsqYjsNGs65g-4iEoGKjpZnlyTgXoHrKFv6MooeOkiucbe8MsIneNQ-vQraaxmJwvFTbsewz_XzIDZlxLWVyCdpzowvoJtlDhZL3XU6MgsuCsGIx1wDVC_Fg_1EpxHxXsvZDch1tnYHPq1QkEuKxn3goHSOUW4plGytXJp8PucPz0IyD8Pke47h0_9ryNgqChFa1ml7xZWTu57A_7UFaarwUUIAPAZaxpzdNw0y8Rz21YJWk32iTOi5t-kYT80XMQXzOvii4pTlov7QW2VQMJ0W_YCbh9bDCnJ7TyMt_SsKKb_KrLBZ6ZaCgFuuKao_0igHQeT6r_tsb5urljjikb2GyDvchuZbl4aPC-B5UnLK9HAyPzdw7hdHiKYyiO2wxKNbk9fDDwI8Gzgvd2TdxXhPIbX-s9SkKIb9B833448J1JXRPDlQvSXpXTfNSNQQlixV-XG1dhIrMSDCKu3iehUPOUXzGguMCTIKz_jVNZDfM5AnbgEaXBqLJDjVrNxNcvQdiH7WWxHKNR-IETCwXVlUMzUXlr4D8QRg9_xHvPF-HvxptVjPIuRmNpdXRnATisl9rjYlgC6j78qNlLrkYBHZVcCadZ87sDI-SyaE1HAE-m_tnLfm0uPFQ_2N9Bb8hG-0qVmmNQmSbfvcI5twSSXSsuTVhJ7thdPwiTLIk23ynGVzFyJJC1ZiiOl3L6T6zYB7EDBS5NoxjXNGqHY05A_xoOlHssvxllKvGP79U1IK2rhsff1WMUcY=w1912-h1566"
alt="HW4"></img>

In [60]:
import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

## Setup seed

In [61]:
def setup_seed(seed):
   torch.manual_seed(seed)
   torch.cuda.manual_seed(seed)
   torch.cuda.manual_seed_all(seed)
   np.random.seed(seed)
   random.seed(seed)
   torch.backends.cudnn.benchmark = False
   torch.backends.cudnn.deterministic = True
   torch.backends.cudnn.enabled = True

setup_seed(8888)

## Data constructing

In [62]:
housing_train_df = pd.read_csv('sample_data/california_housing_train.csv')
housing_test_df = pd.read_csv('sample_data/california_housing_test.csv')

In [63]:
housing_df = pd.concat([housing_train_df, housing_test_df], ignore_index = True)
housing_df = (housing_df - housing_df.mean()) / housing_df.std() 

housing_df.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,2.623352,-0.672591,-1.083095,1.366966,1.771168,-0.362989,-0.072107,-1.251624,-1.213104
1,2.543496,-0.574283,-0.765186,2.303412,3.240388,-0.262197,-0.095684,-1.079867,-1.098875
2,2.498578,-0.906658,-0.924141,-0.880871,-0.865347,-0.965969,-1.002085,-1.168851,-1.050414
3,2.493587,-0.930065,-1.162572,-0.522007,-0.477834,-0.805056,-0.716543,-0.358055,-1.156855
4,2.493587,-0.962834,-0.685709,-0.543603,-0.503985,-0.708685,-0.622235,-1.024614,-1.225219
5,2.488596,-0.934746,0.029585,-0.574389,-0.717949,-0.667131,-0.682487,-0.278017,-1.151662
6,2.488596,-0.944109,-0.288323,0.124039,0.337607,0.367307,0.349657,-0.629004,-1.078971
7,2.483605,-0.372985,0.983311,-0.838597,-0.879611,-0.928835,-0.894679,-1.138646,-1.372333
8,2.483605,-0.944109,0.426971,0.988804,1.514411,1.510494,1.457772,-0.891376,-1.286661
9,2.478614,-0.372985,1.380697,-0.523845,-0.544401,-0.564571,-0.598658,-0.884746,-1.375794


In [64]:
from sklearn.model_selection import train_test_split 

housing_train_df, housing_test_df = train_test_split(housing_df, test_size=0.2, random_state=42)

print(f"Total {len(housing_df)}, Training: {len(housing_train_df)}, Testing: {len(housing_test_df)}")

Total 20000, Training: 16000, Testing: 4000


## Construct Dataset

In [65]:
class HousingDataset(Dataset): # inherit Dataset

  def __init__(self, dataframe):
    self.dataframe = dataframe

  def __getitem__(self, index):
    row = self.dataframe.iloc[index].to_numpy().astype('float32') # using index to locate a data row
    features = row[0:-1] # get all columns except last one
    label = row[-1] # last column => we want to predict
    return features, label

  def __len__(self):
    return len(self.dataframe)

In [66]:
setup_seed(8888)

train_dataset = HousingDataset(housing_train_df.astype('float32'))
test_dataset = HousingDataset(housing_test_df.astype('float32'))

train_batch_size = int(len(train_dataset))
test_batch_size = int(len(test_dataset))
train_dataloader = DataLoader(train_dataset, batch_size = train_batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = test_batch_size, shuffle = True)

In [67]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


## SLFN

In [68]:
class SLFN(torch.nn.Module):

  def __init__(self, D_in, H, D_out, activation_func, initial_weights = None):
    super(SLFN, self).__init__()
    self.fc1 = torch.nn.Linear(D_in, H)
    self.fc2 = torch.nn.Linear(H, D_out)
    self.bn1 = torch.nn.BatchNorm1d(H) # batch normalization => normalize between fc and activation func
    self.dropout = torch.nn.Dropout(p=0.5) # inverted drop out
    
    self.activation_func = activation_func
    if activation_func == 'ReLU':
      self.relu = torch.nn.ReLU()
    if activation_func == 'Tanh':
      self.tanh = torch.nn.Tanh()

    self.initial_weights = initial_weights
    if self.initial_weights == 'small random number':
      # H = W * X + B ; X = D_in => linear algebra
      # D_in (1, D_in), W (D_in, H)
      self.fc1.weight = torch.nn.Parameter(0.01 * torch.FloatTensor(np.random.randn(H, D_in)))
      self.fc2.weight = torch.nn.Parameter(0.01 * torch.FloatTensor(np.random.randn(D_out, H)))
    if self.initial_weights == 'Xavier':
      torch.nn.init.xavier_uniform_(self.fc1.weight)
      torch.nn.init.xavier_uniform_(self.fc2.weight)
    if self.initial_weights == 'Kaiming':
      torch.nn.init.kaiming_uniform_(self.fc1.weight)
      torch.nn.init.kaiming_uniform_(self.fc2.weight)
  
  def forward(self, x): # input
    fc1 = self.fc1(x) 

    # activation
    activation = None
    if self.activation_func == 'ReLU':
      activation = self.relu(fc1)
    if self.activation_func == 'Tanh':
      activation = self.tanh(fc1)
    
    pred = self.fc2(activation) 
    return pred

  # inverted drop out (don't affect at testing phase)
  def forward_do(self, x):
    fc1 = self.fc1(x)
    do1 = self.dropout(fc1)

    # activation
    activation = None
    if self.activation_func == 'ReLU':
      activation = self.relu(fc1)
    if self.activation_func == 'Tanh':
      activation = self.tanh(fc1)

    pred = self.fc2(activation) 
    return pred

  # input -> FC1 -> BN1 -> ReLU -> FC2 -> output
  def forward_bn(self, x): # input
    fc1 = self.fc1(x) 
    bn1 = self.bn1(fc1) 

    # activation
    activation = None
    if self.activation_func == 'ReLU':
      activation = self.relu(bn1)
    if self.activation_func == 'Tanh':
      activation = self.tanh(bn1)
    
    pred = self.fc2(activation) 
    return pred

In [69]:
D_in = len(housing_train_df.columns) - 1 
D_out = 1 
hidden_nodes = [5, 8, 11] 
learning_rate = 1e-4 
loss_fn = torch.nn.MSELoss().to(device) 

## Weight Tuning EB and Regularizing EB
We only show regularizing algorithm </br>
<img
width="600px"
src="https://lh3.googleusercontent.com/fife/APg5EOYOXcEzwpZUa-FLsXeR6yMrJ5rBcorGJm1VsaPjEF5gQYWzduPuyIx57aKhJ0-MY5feL2NlTm-DvXXEqApk9DQ9DBQVM7Ue6urUwcBG9499VknBAfKf18x0JY3mjT2B4pcFqY-zqeUDtj8f3inLFFSmqio1Gwk-IelGawU4YSZYB3DNsJ55OyEH5atm7Qn4_lFxh_79kXGa3SD-j9VQmiMWcJGnf0QxOKWq8D5dmecd5XIldK_xGbIQGug0hObWijKU6Q6C-vTIk8rM85qTx6NEzau-mOKAP0m0BwhcmiwWAUeDzdayUVTJZ6PzyITXbTTbGbJRcVTxzmiYYsaK6tvs1YMdfdJZoLVlRuXRpQLX0BxQrPL0pAxgP3J1ty-WSZXrd1aAcdKgs5TOdHWtkac-NBVTv4MQQYahX7O595IIdkNSawJ0JykDYC9KcY0tKXH9NkKPGq2TwlIn5KHuUYoCz0ztSAIziHF2HqWPqGxHhDnYDG8bgITMRP-P6-czrwPUaFB0FjSq-3KDmfkoCf7kWH7owyhE1aOckgmiw4zdY1n6qXZPEipj5WD_hSG2zoibeuvdVGZmbasAehh6J7RHKQn1U4Q09cBN15Z2Vkl6xulV2rR6poGV5NIft5YuOJIwNvRSuAaEFzwGN8vpapqTKdcCbrrAfD7u4VHD1Wnej00pbwNZQKzJRWVrP0BvySR2uOu696M3IvtTW8bPNMMqECpIMX72qqRGtofr_emyoPr_YQMt0dDuZ7jYpFzI3CFAWRpmXLdfAAHnfuuLtaOYhv6rD-2DoHJOCTelW2oXx614DHH2e03qD_fGhLpZ3aCAUGGFKhdUvRWoS0aWCx4p4UbW1SLTanmuxdqVue8QyQY9B6Bpxl1QeQe3ZvbuzFskY4IDPlP8AvI13QEN4d8CcMEhZH7155KmBPUGGpZGekw2xx0Ns5KNyB5mXIjVnIraYwhW1fX8rbLdvKcWw53Mpeo2BsS5STdIKEQjNEr1xT54o0giiQmKiRIklDnYOi8HPaRQ_WK-icof1dlTCFUKd9KBwnzQRp-be_N-X1F5y05oABPq0Egb2yF_OIPXhVbUWYKw4OC8fBYbrnG9TgXfMwL26Su5AvKH9l2L2_wwMD4NCtUbSlck1T9OWWhhow8XD-4LeOULKNp5mjEZy_2dJvD57dtoU2KIrrMM8KjD02Ujf2ZVHPc_9fdhmF_l-0tT0T6DsMr_dU6xNU95JIGgm-prmRbP28LHLPrhweASm8cljKgynbYqJvZueKDRq6Z7AwTiVlH3A3M-ZiMCzEIMit9mDcn1-ms9_HBNRa8p-h-ZGQMw9v7r7Zit1l3XIeHu_9rZb2BzLHNKLvFDgQVE4c_VuIJC7VsI1iLp5313UgOBrgSfEmgNBNsAX7TYJSOiPxMbvqY5F63AGWL790UNWeXWNKDINR6NI6WcApe-OTeM2T-d4J8BkRA3fcOulP-m4Yjmd6uqulUZKx3BVGjOfFlzyqsFdsA8bcTYb8RuM9SNs4jxHIlG4z-ssDE6mvMevmBJLbepIIxOW1HcPmWWg9VSe8nQs9zMbQUH6rqhgb-KklaUhS8_6XrbcGgLJXc=w1466-h1622"></img>

### Weight Tuning

In [70]:
setup_seed(8888)
epoch_bound = 100 # EB
epsilon = 0.995 # LG

model_EB_LG = SLFN(D_in, hidden_nodes[1], D_out, 'ReLU', 'small random number').to(device)
optimizer_EB_LG = torch.optim.Adam(model_EB_LG.parameters(), lr = learning_rate) 

In [59]:
t = 0
loss = np.inf

while t < epoch_bound and loss > epsilon:
  for batch, (X, y) in enumerate(train_dataloader): 
    # forward 
    pred = model_EB_LG(X.to(device))
    loss = loss_fn(pred, y.to(device))

    # backward
    loss.backward()
    optimizer_EB_LG.step()
    optimizer_EB_LG.zero_grad()

  if t % 10 == 0:
    print(f"{t}")
    print(f"loss: {loss}")
    print("----------------------")

  t+=1

print(f"{t}th epoch")
print(f"loss: {loss}")
if loss < epsilon:
  print("Accetable SLFN!")
else:
  print("Unaccetable SLFN!")

0
loss: 0.9986434578895569
----------------------
10
loss: 0.9983462691307068
----------------------
20
loss: 0.9980566501617432
----------------------
30
loss: 0.9977754950523376
----------------------
40
loss: 0.9975031614303589
----------------------
50
loss: 0.9972395896911621
----------------------
60
loss: 0.9969844818115234
----------------------
70
loss: 0.9967378973960876
----------------------
80
loss: 0.9964994788169861
----------------------
90
loss: 0.9962692260742188
----------------------
100th epoch
loss: 0.9960691332817078
Unaccetable SLFN!


### Regularizing - Epoch Bound

In [71]:
setup_seed(8888)

# store weight first
original_param = {}
for param in model_EB_LG.parameters():
  original_param[param] = param

original_param

{Parameter containing:
 tensor([[-4.1122e-03, -4.9928e-04,  1.8260e-03,  2.4875e-02,  1.7346e-03,
          -1.1060e-02, -6.0659e-03,  9.4524e-04],
         [ 1.1628e-02,  2.7409e-03,  1.3594e-02,  7.5668e-03, -1.7076e-02,
           2.4890e-03,  1.0393e-02,  7.6131e-03],
         [-1.0846e-02,  1.9589e-03, -1.1032e-02,  1.0573e-02,  1.6618e-02,
          -7.0028e-03,  1.0204e-03, -4.5638e-03],
         [-1.0389e-02, -2.1280e-02,  5.9019e-03, -8.4633e-03, -7.5374e-03,
           1.3749e-03,  6.5155e-03, -4.1925e-03],
         [-7.7539e-03,  5.6415e-03, -7.1718e-03, -1.2726e-02, -2.4334e-03,
           7.9423e-03,  1.0855e-03,  1.2128e-02],
         [-1.5914e-02, -1.1129e-02, -3.4936e-02, -1.3797e-02,  4.1254e-03,
          -4.8289e-03,  1.0431e-02,  1.2035e-02],
         [-8.8780e-03,  8.2943e-03, -1.0354e-02, -9.1499e-03, -3.8567e-03,
           1.4116e-02, -6.6725e-03,  4.8900e-05],
         [ 2.5719e-02,  1.3290e-02,  5.1604e-03, -3.4849e-03, -8.3781e-03,
           8.0108e-03, -9.7

In [72]:
# define loss function with regularization term (L2 regularizing)
# using weight decay in pytorch
# or simply add regularization term in training phase
# tutorial: https://androidkt.com/how-to-add-l1-l2-regularization-in-pytorch-loss-function/
optimizer_EB_r = torch.optim.Adam(model_EB_LG.parameters(), lr = learning_rate, weight_decay=0.001) 

In [73]:
for i in range(epoch_bound):
  for batch, (X, y) in enumerate(train_dataloader):
    pred = model_EB_LG(X.to(device))
    loss = loss_fn(pred, y.to(device))
    loss.backward()
    optimizer_EB_r.step()
    optimizer_EB_r.zero_grad()

  if i%10 == 0:
    print(f"{i}")
    print(f"loss: {loss}")
    print("----------------------")

# check if loss become smaller
for batch, (X, y) in enumerate(train_dataloader):
  pred = model_EB_LG(X.to(device))
  loss = loss_fn(pred, y.to(device))

  print(f"loss: {loss}")
  if loss < epsilon:
    print("Acceptable SLFN!")
  else:
    # restore w
    for param in model_EB_LG.parameters():
        param = original_param[param]

    print("Unacceptable SLFN!")

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16000])) that is different to the input size (torch.Size([16000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0
loss: 0.9986435174942017
----------------------
10
loss: 0.9983468055725098
----------------------
20
loss: 0.9980581998825073
----------------------
30
loss: 0.9977777600288391
----------------------
40
loss: 0.997505784034729
----------------------
50
loss: 0.9972425699234009
----------------------
60
loss: 0.9969879984855652
----------------------
70
loss: 0.9967420697212219
----------------------
80
loss: 0.9965041279792786
----------------------
90
loss: 0.9962746500968933
----------------------
loss: 0.9960530996322632
Unacceptable SLFN!


In [74]:
[para for para in model_EB_LG.parameters()]

[Parameter containing:
 tensor([[-5.8515e-04, -4.0529e-04,  5.3400e-04,  1.5782e-02, -1.5714e-04,
          -3.0158e-03, -2.5888e-04, -1.6293e-03],
         [ 2.9953e-03, -3.6691e-04,  5.4048e-03, -1.0521e-03, -8.5010e-03,
          -3.4984e-03,  2.1135e-03, -7.4957e-04],
         [-2.3132e-02,  1.6143e-02, -2.4707e-02,  2.3572e-02,  2.9644e-02,
           5.7535e-03,  1.3878e-02,  9.4470e-03],
         [-3.5210e-03, -1.2071e-02,  9.8798e-04, -3.0500e-03, -1.1508e-03,
          -3.9110e-03, -1.7468e-03, -4.0492e-03],
         [-4.0122e-03,  5.3402e-04,  1.4281e-03, -7.1565e-03,  1.9483e-03,
           7.4330e-04,  3.7719e-03,  3.2281e-03],
         [-7.2030e-03, -3.1051e-03, -2.5467e-02, -5.3232e-03, -3.0235e-05,
           4.5297e-05,  2.5692e-03,  3.8326e-03],
         [-1.4798e-03,  1.1196e-03, -2.5118e-03, -1.6580e-03,  1.2815e-05,
           5.6011e-03, -3.2717e-04,  2.2905e-07],
         [ 1.6156e-02,  5.1461e-03,  1.0652e-03, -1.8373e-03, -1.7909e-03,
          -2.4108e-03, -2.8

## Weight Tuning EB and Regularizing DO
<img
width="600px"
src="https://lh3.googleusercontent.com/fife/APg5EOYC0ll8hGHIiZfY_M_LwGljkHaznZiy3NsR3eRLTTBzLN2iTVLRbjYNC7qYTde627GD7FFL8pbEvIqt7SAfxPYLJcEmjQTlLCLE21cH1Bx9uX-pJs-qqr4RmHYj-b5hFAxOCf7FNXyxMzhLnMW1k9CH6ifMFhSe01Eh_pO3Wznx3ThwsTugJxCU9WwBhbL88QMl_U0tYayVLvrM7k5L8SbKT7nr7ZdwEmnJXqLBeNgyW21AjsCH10zVBLn09fsh0M5O-teFgbj2ubcARzwXErmYXGf0LXVpPZOdD_5O--ennLAvE72SNqeMBSpxSh-t9g9jVmANcxQrLZsOMyLWe3IAiI8ksDYmHYwY7VetKLivIq3vEdlHka4jXx0R0rW2AYVo__L5g086QU3SQQ9lXNMhbYJf8SaVIzImFoCslr03olYFKn5ZDSTW-Od-c4B3RHxrlIwBJ8or-XLwtPpWMRypuMQzYLEXtjXXNmKxDof7UsLvDjHVMaTAkqhU762Xqs4pIHtL55sC2kE6WA-SlKbBR_EU2_nmX8EgfJXGSbppeGud5KYXFZOrOvrDLo1mNZyObCxpauiI8Kp_zrmvbs6qHvKhm5ZuEIan0EX-pqggdGO6DOnsnqv0a4W9NICoJSQJmTBUipEmQ04gNvUsbNcHAsf4julAFwJF9nMC4nE4wHX0RX2geFgl-N3UgvGluTDpC0_MUEO9242g0BBp9YN42xO1gUcOI57AJeaLFRQlhs3IIe6OUGxMwhojnPR_0hhHrivsJnlHOKqQ3Dk4fJ5-MWN5r7YMatlu3f8Dcgb9FDu2pdiabiBPE089mo_nhqKGuI4szwGvA21uIVHXEqZoBW06Kz1BmDO32JzC07tldQmU3jHt43qK7qh-bXzlkcRMsbl3hxTxLZXu6ilBPSbaYJOXWKZy7jknNajypqAbvEmLkrtdsy3f6tS1KM5tcT4muvg5Yt_TyXroChEgwp805NCRXGK4ipQ5hT5sXF1sUh9xsACPfepTbRTQI8pqJ6boSMWW2nGihFd22TDNc7gts_WluF2je2H7iAtT4f7Sq6pL0FxS_n8WfVeY9EXy1Dm9G7X2Iz1JrOl40N7QOhHR-uDCk8gvEJ6YGWCa5r2OBxFEl6dbOnfrFKRwdpRO58JuDbto9bp0vmEs3WSK65EaVQMvVOuKquQ6KRTzpMaWH_HwsdHqxPUZP9p8DVIizcmCSeUUMboNuvN-sw_Ko0a5-EtHDKPzORaHkrSIjHHz8DPWifbX3y4xL98mtrwyAqJFmDwee0k-zK5NtXvif-sABZun7s9_mk4-T4jorB5TOvEZ_UrkcuhiVW7TeyoNgQtrRoG5wBMQe6jc8tczU7-9rVtawDgSs8GahrHLZVFzk8nzBSXRlkN8Jm_K1nCZPu3is7-CWWciIDWqYhhhVk04PwfA2qSfzYbCNZkfPULQUNBE531LxJTlRJLD560FIVntJr5lJAuZYhuVE_qkf1yUqQPaSrLN44k9LeulwSls01gISswztJqRa-g26Tcn-UroOY1794CPJw4Xiy4OEaFEu-tIGy9-guOLGmjHRlrSt9xeR5By37_4puruntPepxo=w1466-h1654"></img>

### Regularizing - Dropout

In [75]:
# use forward dropout mode
for i in range(epoch_bound):
  for batch, (X, y) in enumerate(train_dataloader):
    pred = model_EB_LG.forward_do(X.to(device))
    loss = loss_fn(pred, y.to(device))
    loss.backward()
    optimizer_EB_LG.step()
    optimizer_EB_LG.zero_grad()

  if i%10 == 0:
    print(f"{i}")
    print(f"loss: {loss}")
    print("----------------------")

# check if loss become smaller
for batch, (X, y) in enumerate(train_dataloader):
  pred = model_EB_LG(X.to(device)) # still use normal dropout => inverted dropout
  loss = loss_fn(pred, y.to(device))

  print(f"loss: {loss}")
  if loss < epsilon:
    print("Acceptable SLFN!")
  else:
    # restore w
    for param in model_EB_LG.parameters():
        param = original_param[param]

    print("Unacceptable SLFN!")

0
loss: 0.996053159236908
----------------------
10
loss: 0.9958193898200989
----------------------
20
loss: 0.9955945014953613
----------------------
30
loss: 0.9953791499137878
----------------------
40
loss: 0.9951740503311157
----------------------
50
loss: 0.9949794411659241
----------------------
60
loss: 0.9947953820228577
----------------------
70
loss: 0.9946216344833374
----------------------
80
loss: 0.9944584369659424
----------------------
90
loss: 0.9943054914474487
----------------------
loss: 0.9941623210906982
Acceptable SLFN!


In [77]:
[para for para in model_EB_LG.parameters()]

[Parameter containing:
 tensor([[ 8.7154e-04, -1.0713e-02, -9.1022e-03,  6.4833e-03,  8.2373e-03,
          -6.5233e-03,  8.3411e-03,  2.8763e-03],
         [ 4.1791e-03, -1.0149e-02, -3.7945e-03, -9.5022e-03, -6.1113e-04,
          -6.7911e-03,  1.0218e-02,  3.0127e-03],
         [-3.3584e-02,  2.6573e-02, -3.4950e-02,  3.3712e-02,  3.9762e-02,
           1.5879e-02,  2.4004e-02,  2.0204e-02],
         [-1.6503e-02, -1.3180e-02,  1.2855e-02, -1.7293e-02, -1.5532e-02,
          -1.8041e-02, -1.6131e-02, -1.3932e-02],
         [-3.1573e-03, -8.7334e-03, -7.3155e-03, -1.4837e-02,  9.3486e-03,
          -2.3929e-03,  1.1393e-02,  6.2914e-03],
         [-7.2030e-03, -3.1051e-03, -2.5467e-02, -5.3232e-03, -3.0235e-05,
           4.5297e-05,  2.5692e-03,  3.8326e-03],
         [-1.4798e-03,  1.1196e-03, -2.5118e-03, -1.6580e-03,  1.2815e-05,
           5.6011e-03, -3.2717e-04,  2.2905e-07],
         [ 3.9446e-03,  2.9420e-03,  1.2466e-02, -1.6105e-02, -1.5797e-02,
          -1.6670e-02, -1.6

## Weight Tuning EB and Regularizing BN
<img
width="600px"
src="https://lh3.googleusercontent.com/fife/APg5EOZWqgUCaWuJEXYB_u2gNydfsuVESMdw8e7xDekXvUvJadbuatMTygf8vaCge_kQMijMXh4eQ8s_YXeoCZLWIQb3v7-uNr3CEeaVzgVgml4HVskAkGIedo5D6UawqAwj5h3JIYjvEXFrDT1PuLoXaLYYH3cdUsXUmCaIQEvGq4LUIsaieIxBeK7IUNbxUo0kHP4G0DIyw6r4FVS47eB4qZ6ld0vi-oztLmsgdCvLoTVWuEMczjDSqDkTYFVcXyAz1V36teZu_X_tldYGuPPiak_TLAhSxdPPRUmn6Q4uk3RHy8LLpsVdxQprGLscrfhyooRzpaPrvbww3aWvlAvvJIKgpJrA-i2VEqZ80_uzy63K2QCXtCrf8nB1qzpoZ0BkaNFsR861jQPIBVPBc_3VHu6x5uS1krCGYGJ0gOYVUanYifwMNf3_0axjXLMTGKlXT3dv--f43iKS42u_CCdOPS-m-L-1uWQZ9MnIGfaFy_royFQJfRN9IBg1sLAqqVWEaljUN-0v5gmCSIK_JZofJ_uKZqzlYeXt39mnS5j1uyTDstR4dCtUK_9_nj2K1gs0BvNDI84lIkp-jTwfJY1GFYsLkGqk-IT8QwxyIH3nUZRf1tjjMf2Otlhjqk6tnQo0WGEsljXFqcaIb_2GCjeFypoIYaiHaBM9-FjXu9JfU084LMJ88ZsKLrZs7f67_xYdi07jJkvBm4AbWPpu93bE0krhebc5MHo3xDOYwzM5D_E4Z8IbISLo-aOwOLkgAkEFhMg7IFW7XDlzBT61jxZXCIMnElT7derFfOcQl5eFmF8JSPtXAlys_pVDJJSedoZkCOZDQZwERR_kLOd6-IOFhUMSJUxX3TxmTuy3iFiqxR1ih3dblXtyAqFZ_leH3Nd5kyuW15W36PONtfHlH_y3zS4T2JaSAaGrkHhayvPccmxATRuR7yuM0Z9_milcDzrEyyjUv3ZtYHn2EZvE7p0DPClLWXx3IPzSnn0L9Aa5breswa1R3ouQas3Eg6jzbrourM8dU5uo8AVtVaQ5BdZT9LrFupzEe0j6m6bdbmjKOkj6WZBMEG2dk36Bp7VZJFArK96gkwt21VzxfkzztdC0pQir5lvFb3sSqdM4ktIfM9SFq7XksVKQIccWObhF-oyBJ-AGFMh4kRojcXtT6vcyIPwyKBhp2IXtbnFay1SOtydxoTFGcrbw5TUbkgj9TyT575XshKQONM_BD0N-T3H2sTVsR3dTIWgDzVWz1fvyEPSkuGUrfBuq6jfPEas5qRt6dDka75mOck6uN1NzaVnLZtb4rJWbbsn78S8bnmvID0DJqqgay7t2-IJhhFtbDUW_0uixjxgnYMrWIUPUEeZy8o89T5Ah5In_wMzn4qQCF0u0XJuuDx78JhzU0Qm5Tibb7Z7izv5HPCeRJDFmOpyVRogBR63j6yNDNMx9r2hm4gSSqwUB9SiZst5HSR1dJmvd60v1EZTS6b5GX9DpOBrcrQUSVTNS_cCNBqkTM0yCZZgIWgeLBrLgfH3np3JHjAtwRtsbG0w_UorOSeJkfLeMOCyg3u2McjXu5rGtIH5tNZzYsXofc_XzKRPCFXZKIFr6E6o=w1466-h1654"></img>

### Regularizing - Batch Normalization

In [80]:
# use original model
# use forward dropout mode
for i in range(epoch_bound):
  for batch, (X, y) in enumerate(train_dataloader):
    pred = model_EB_LG.forward_bn(X.to(device))
    loss = loss_fn(pred, y.to(device))
    loss.backward()
    optimizer_EB_LG.step()
    optimizer_EB_LG.zero_grad()

  if i%10 == 0:
    print(f"{i}")
    print(f"loss: {loss}")
    print("----------------------")

# check if loss become smaller
for batch, (X, y) in enumerate(train_dataloader):
  pred = model_EB_LG(X.to(device)) # still use normal dropout => inverted dropout
  loss = loss_fn(pred, y.to(device))

  print(f"loss: {loss}")
  if loss < epsilon:
    print("Acceptable SLFN!")
  else:
    # restore w
    for param in model_EB_LG.parameters():
        param = original_param[param]

    print("Unacceptable SLFN!")

0
loss: 0.993184506893158
----------------------
10
loss: 0.9931737780570984
----------------------
20
loss: 0.9931650161743164
----------------------
30
loss: 0.9931573867797852
----------------------
40
loss: 0.9931507706642151
----------------------
50
loss: 0.9931449294090271
----------------------
60
loss: 0.9931395053863525
----------------------
70
loss: 0.9931345582008362
----------------------
80
loss: 0.993129551410675
----------------------
90
loss: 0.9931249618530273
----------------------
loss: 0.9938995242118835
Acceptable SLFN!


In [81]:
[para for para in model_EB_LG.parameters()]

[Parameter containing:
 tensor([[ 0.0093, -0.0196, -0.0026,  0.0021,  0.0002, -0.0010,  0.0003, -0.0056],
         [ 0.0134, -0.0190,  0.0058, -0.0079, -0.0021,  0.0025,  0.0089, -0.0037],
         [-0.0417,  0.0346, -0.0541,  0.0237,  0.0270,  0.0053,  0.0112,  0.0211],
         [-0.0291, -0.0030,  0.0091, -0.0031, -0.0058, -0.0009, -0.0011,  0.0003],
         [ 0.0059, -0.0202, -0.0037, -0.0156,  0.0056,  0.0006,  0.0082,  0.0153],
         [ 0.0032, -0.0126, -0.0079, -0.0014,  0.0073,  0.0014,  0.0098,  0.0125],
         [ 0.0033, -0.0032, -0.0024,  0.0006,  0.0047,  0.0026,  0.0040,  0.0077],
         [-0.0032,  0.0124,  0.0089, -0.0158, -0.0118, -0.0011, -0.0114, -0.0236]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.1731,  0.3173, -0.0157,  0.2039,  0.1558, -0.3325, -0.2151,  0.1937],
        requires_grad=True),
 Parameter containing:
 tensor([[ 0.0138,  0.0210,  0.0299,  0.0217,  0.0319, -0.0083, -0.0096,  0.0232]],
        requires_grad=True),
 Parameter co

In [ ]:
torch.save(model_EB_LG, 'model_EB_LG.pt')